# The simplest case, running using NativeExecutionEngine

In [1]:
%%fsql
CREATE [[0]] SCHEMA a:int
PRINT

,a:int
0,0


# Test %%fsql parsing

In [2]:
from fugue_jupyter.utils import _FugueSQLMagics
from fugue import register_execution_engine, register_default_execution_engine
from fugue import NativeExecutionEngine
from pytest import raises


class MockEngine(NativeExecutionEngine):
    def __init__(self, conf=None):
        super().__init__(conf=conf)
        
def test_fugue_sql_magic():
    register_execution_engine("m", lambda conf, **kwargs: MockEngine(conf=conf))
    m = _FugueSQLMagics(None, {"a": 1}, {"b": 2})
    e = m.get_engine("   ", {})
    assert isinstance(e, NativeExecutionEngine)
    assert 1 == e.conf["a"]
    assert 2 == e.conf["b"]

    e = m.get_engine(' {"a":2,"c":3, "b":2}  ', {})
    assert isinstance(e, NativeExecutionEngine)
    assert 2 == e.conf["a"]
    assert 2 == e.conf["b"]
    assert 3 == e.conf["c"]

    with raises(ValueError):
        e = m.get_engine(' {"a":2,"b":1}  ', {})

    e = m.get_engine(" m   ", {})
    assert isinstance(e, MockEngine)
    assert 1 == e.conf["a"]
    assert 2 == e.conf["b"]

    e = m.get_engine(' m  {"a":2,"c":3, "b":2}  ', {})
    assert isinstance(e, MockEngine)
    assert 2 == e.conf["a"]
    assert 2 == e.conf["b"]
    assert 3 == e.conf["c"]

test_fugue_sql_magic()

# Run with a different execution engine

This engine is defined in previous test

In [3]:
%%fsql m
CREATE [[0]] SCHEMA a:int
PRINT

,a:int
0,0


# Run with a different SQL engine

In [4]:
%%fsql native+sqlite
CREATE [[0]] SCHEMA a:int
PRINT

,a:int
0,0


# Communicate between fsql cells

You must use `yield` to communicate betweem cells. That means all the intermediate data will be stored in file system. So you should also specify `fugue.workflow.checkpoint.path`.

In [5]:
%%fsql native
a = CREATE [[0]] SCHEMA a:int YIELD DATAFRAME

In [6]:
%%fsql native
SELECT * FROM a
PRINT

ANTLR runtime and generated code versions disagree: 4.11.1!=4.10.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.10.1


,a:int
0,0


In [7]:
from fugue_notebook import setup

setup(fsql_ignore_case=True)

/usr/local/lib/python3.8/site-packages/fugue_sql/__init__.py:8: UserWarning: fsql and FugueSQLWorkflow now should be imported directly from fugue, fugue_sql will be removed in 0.9.0
  warnings.warn(


<IPython.core.display.Javascript object>

In [8]:
%%fsql native
select * from a
print

ANTLR runtime and generated code versions disagree: 4.11.1!=4.10.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.10.1


,a:int
0,0
